# Linear Regression in statsmodels (practice)</h1>

<div><h3></h3></div><div><h3></h3><h3>Task: Run a linear regression with statsmodels predict the price of a car.</h3><p>Note: you will be revisiting this task for a later assignment, so you will want to create a new GitHub repository for your work.</p><ul><li>Dataset&nbsp;Link:&nbsp;&nbsp;<a href="https://www.kaggle.com/datasets/hellbuoy/car-price-prediction" target="_blank">https://www.kaggle.com/datasets/hellbuoy/car-price-prediction</a>&nbsp;<ul><li>&nbsp;Download and save in repo,</li></ul></li></ul><h2>Steps to Perform:</h2><ul><li>&nbsp; &nbsp; Perform initial preprocessing of data</li><li>&nbsp; &nbsp; Perform preprocessing for statsmodels</li><li>&nbsp; &nbsp; Run the model in statsmodels and produce a results summary</li><li>&nbsp; &nbsp; Evaluate the model on the test set</li></ul></div><p><br></p><p>Submit your assignment repository link.&nbsp;</p>
        
        
        </div></div>

# SOLUTION

In [25]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
plt.style.use(['seaborn-muted','fivethirtyeight'])
## Standard Statsmodels import

import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

plt.rcParams['figure.facecolor'] = 'white'
plt.style.use(['seaborn-talk'])

In [2]:
pd.set_option('display.max_columns',100)
df = pd.read_csv("Data/CarPrice_Assignment.csv", index_col=0)
df

,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,carheight,curbweight,enginetype,cylindernumber,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
car_ID,,,,,,,,,,,,,,,,,,,,,,,,,
1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,-1,volvo 145e (sw),gas,std,four,sedan,rwd,front,109.1,188.8,68.9,55.5,2952,ohc,four,141,mpfi,3.78,3.15,9.5,114,5400,23,28,16845.0
202,-1,volvo 144ea,gas,turbo,four,sedan,rwd,front,109.1,188.8,68.8,55.5,3049,ohc,four,141,mpfi,3.78,3.15,8.7,160,5300,19,25,19045.0
203,-1,volvo 244dl,gas,std,four,sedan,rwd,front,109.1,188.8,68.9,55.5,3012,ohcv,six,173,mpfi,3.58,2.87,8.8,134,5500,18,23,21485.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 205 entries, 1 to 205
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   symboling         205 non-null    int64  
 1   CarName           205 non-null    object 
 2   fueltype          205 non-null    object 
 3   aspiration        205 non-null    object 
 4   doornumber        205 non-null    object 
 5   carbody           205 non-null    object 
 6   drivewheel        205 non-null    object 
 7   enginelocation    205 non-null    object 
 8   wheelbase         205 non-null    float64
 9   carlength         205 non-null    float64
 10  carwidth          205 non-null    float64
 11  carheight         205 non-null    float64
 12  curbweight        205 non-null    int64  
 13  enginetype        205 non-null    object 
 14  cylindernumber    205 non-null    object 
 15  enginesize        205 non-null    int64  
 16  fuelsystem        205 non-null    object 
 1

In [4]:
## Drop carNmae
df = df.drop(columns='CarName')


In [5]:
## Make x and y variables
y = df['price'].copy()
X = df.drop(columns=['price']).copy()


X_train,X_test, y_train, y_test = train_test_split(X,y, random_state=321)
X_train.head()

,symboling,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,carheight,curbweight,enginetype,cylindernumber,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg
car_ID,,,,,,,,,,,,,,,,,,,,,,,
18,0,gas,std,four,sedan,rwd,front,110.0,197.0,70.9,56.3,3505,ohc,six,209,mpfi,3.62,3.39,8.0,182,5400,15,20
40,0,gas,std,four,sedan,fwd,front,96.5,175.4,65.2,54.1,2304,ohc,four,110,1bbl,3.15,3.58,9.0,86,5800,27,33
41,0,gas,std,four,sedan,fwd,front,96.5,175.4,62.5,54.1,2372,ohc,four,110,1bbl,3.15,3.58,9.0,86,5800,27,33
48,0,gas,std,four,sedan,rwd,front,113.0,199.6,69.6,52.8,4066,dohc,six,258,mpfi,3.63,4.17,8.1,176,4750,15,19
124,-1,gas,std,four,wagon,fwd,front,103.3,174.6,64.6,59.8,2535,ohc,four,122,2bbl,3.35,3.46,8.5,88,5000,24,30


In [6]:
## make cat selector and using it to save list of column names
cat_select = make_column_selector(dtype_include='object')
cat_cols = cat_select(X_train)

## make num selector and using it to save list of column names
num_select = make_column_selector(dtype_include='number')
num_cols = num_select(X_train)

## make pipelines
cat_pipe = make_pipeline(SimpleImputer(strategy='constant',
                                       fill_value='MISSING'),
                         OneHotEncoder(handle_unknown='ignore', sparse=False)
                        )

num_pipe = make_pipeline(SimpleImputer(strategy='mean'))

preprocessor = make_column_transformer( (num_pipe, num_cols),
                                       (cat_pipe,cat_cols),
#                                        remainder='passthrough'
                                      )
preprocessor

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer())]),
                                 ['symboling', 'wheelbase', 'carlength',
                                  'carwidth', 'carheight', 'curbweight',
                                  'enginesize', 'boreratio', 'stroke',
                                  'compressionratio', 'horsepower', 'peakrpm',
                                  'citympg', 'highwaympg']),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(fill_value='MISSING',
                                                                strategy='constant')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                

In [15]:
## fit the col transformer to learn feature names 
preprocessor.fit(X_train)

## Now create list of our final features after preprocessing
final_features = []


## adding the numeric features which process first in the Col Trans
final_features.extend(num_cols)


## Now, get OHe feature names
cat_features = preprocessor.named_transformers_['pipeline-2'][-1].get_feature_names_out(cat_cols)
final_features.extend(cat_features)


## Transform X vars and remake as dataframes
X_train_tf = pd.DataFrame( preprocessor.transform(X_train), 
                         columns=final_features, 
                         index=X_train.index)

X_test_tf = pd.DataFrame( preprocessor.transform(X_test), 
                         columns=final_features, 
                         index=X_test.index)
X_train_tf.head()

,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,fueltype_diesel,fueltype_gas,aspiration_std,aspiration_turbo,doornumber_four,doornumber_two,carbody_convertible,carbody_hardtop,carbody_hatchback,carbody_sedan,carbody_wagon,drivewheel_4wd,drivewheel_fwd,drivewheel_rwd,enginelocation_front,enginetype_dohc,enginetype_dohcv,enginetype_l,enginetype_ohc,enginetype_ohcf,enginetype_ohcv,enginetype_rotor,cylindernumber_eight,cylindernumber_five,cylindernumber_four,cylindernumber_six,cylindernumber_three,cylindernumber_twelve,cylindernumber_two,fuelsystem_1bbl,fuelsystem_2bbl,fuelsystem_4bbl,fuelsystem_idi,fuelsystem_mfi,fuelsystem_mpfi,fuelsystem_spdi,fuelsystem_spfi
car_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
18,0.0,110.0,197.0,70.9,56.3,3505.0,209.0,3.62,3.39,8.0,182.0,5400.0,15.0,20.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
40,0.0,96.5,175.4,65.2,54.1,2304.0,110.0,3.15,3.58,9.0,86.0,5800.0,27.0,33.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41,0.0,96.5,175.4,62.5,54.1,2372.0,110.0,3.15,3.58,9.0,86.0,5800.0,27.0,33.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
48,0.0,113.0,199.6,69.6,52.8,4066.0,258.0,3.63,4.17,8.1,176.0,4750.0,15.0,19.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
124,-1.0,103.3,174.6,64.6,59.8,2535.0,122.0,3.35,3.46,8.5,88.0,5000.0,24.0,30.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
## Adding contstants for statsmodels
X_train_df = sm.add_constant(X_train_tf, prepend=False)
X_train_df

,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,fueltype_diesel,fueltype_gas,aspiration_std,aspiration_turbo,doornumber_four,doornumber_two,carbody_convertible,carbody_hardtop,carbody_hatchback,carbody_sedan,carbody_wagon,drivewheel_4wd,drivewheel_fwd,drivewheel_rwd,enginelocation_front,enginetype_dohc,enginetype_dohcv,enginetype_l,enginetype_ohc,enginetype_ohcf,enginetype_ohcv,enginetype_rotor,cylindernumber_eight,cylindernumber_five,cylindernumber_four,cylindernumber_six,cylindernumber_three,cylindernumber_twelve,cylindernumber_two,fuelsystem_1bbl,fuelsystem_2bbl,fuelsystem_4bbl,fuelsystem_idi,fuelsystem_mfi,fuelsystem_mpfi,fuelsystem_spdi,fuelsystem_spfi
car_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
18,0.0,110.0,197.0,70.9,56.3,3505.0,209.0,3.62,3.39,8.0,182.0,5400.0,15.0,20.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
40,0.0,96.5,175.4,65.2,54.1,2304.0,110.0,3.15,3.58,9.0,86.0,5800.0,27.0,33.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41,0.0,96.5,175.4,62.5,54.1,2372.0,110.0,3.15,3.58,9.0,86.0,5800.0,27.0,33.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
48,0.0,113.0,199.6,69.6,52.8,4066.0,258.0,3.63,4.17,8.1,176.0,4750.0,15.0,19.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
124,-1.0,103.3,174.6,64.6,59.8,2535.0,122.0,3.35,3.46,8.5,88.0,5000.0,24.0,30.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,-1.0,109.1,188.8,68.9,55.5,2952.0,141.0,3.78,3.15,9.5,114.0,5400.0,23.0,28.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
42,0.0,96.5,175.4,65.2,54.1,2465.0,110.0,3.15,3.58,9.0,101.0,5800.0,24.0,28.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
32,2.0,86.6,144.6,63.9,50.8,1819.0,92.0,2.91,3.41,9.2,76.0,6000.0,31.0,38.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
X_test_df = sm.add_constant(X_test_tf, prepend=False)
display(X_train_df.head(2), X_test_df.head(2))

,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,fueltype_diesel,fueltype_gas,aspiration_std,aspiration_turbo,doornumber_four,doornumber_two,carbody_convertible,carbody_hardtop,carbody_hatchback,carbody_sedan,carbody_wagon,drivewheel_4wd,drivewheel_fwd,drivewheel_rwd,enginelocation_front,enginetype_dohc,enginetype_dohcv,enginetype_l,enginetype_ohc,enginetype_ohcf,enginetype_ohcv,enginetype_rotor,cylindernumber_eight,cylindernumber_five,cylindernumber_four,cylindernumber_six,cylindernumber_three,cylindernumber_twelve,cylindernumber_two,fuelsystem_1bbl,fuelsystem_2bbl,fuelsystem_4bbl,fuelsystem_idi,fuelsystem_mfi,fuelsystem_mpfi,fuelsystem_spdi,fuelsystem_spfi
car_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
18,0.0,110.0,197.0,70.9,56.3,3505.0,209.0,3.62,3.39,8.0,182.0,5400.0,15.0,20.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
40,0.0,96.5,175.4,65.2,54.1,2304.0,110.0,3.15,3.58,9.0,86.0,5800.0,27.0,33.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,fueltype_diesel,fueltype_gas,aspiration_std,aspiration_turbo,doornumber_four,doornumber_two,carbody_convertible,carbody_hardtop,carbody_hatchback,carbody_sedan,carbody_wagon,drivewheel_4wd,drivewheel_fwd,drivewheel_rwd,enginelocation_front,enginetype_dohc,enginetype_dohcv,enginetype_l,enginetype_ohc,enginetype_ohcf,enginetype_ohcv,enginetype_rotor,cylindernumber_eight,cylindernumber_five,cylindernumber_four,cylindernumber_six,cylindernumber_three,cylindernumber_twelve,cylindernumber_two,fuelsystem_1bbl,fuelsystem_2bbl,fuelsystem_4bbl,fuelsystem_idi,fuelsystem_mfi,fuelsystem_mpfi,fuelsystem_spdi,fuelsystem_spfi,const
car_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
49,0.0,113.0,199.6,69.6,52.8,4066.0,258.0,3.63,4.17,8.1,176.0,4750.0,15.0,19.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
197,-2.0,104.3,188.8,67.2,56.2,2935.0,141.0,3.78,3.15,9.5,114.0,5400.0,24.0,28.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [18]:
## Fit OLS Model 
model = sm.OLS(y_train,X_train_df)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.933
Model:                            OLS   Adj. R-squared:                  0.909
Method:                 Least Squares   F-statistic:                     38.91
Date:                Sat, 07 May 2022   Prob (F-statistic):           1.59e-49
Time:                        18:56:06   Log-Likelihood:                -1379.1
No. Observations:                 153   AIC:                             2840.
Df Residuals:                     112   BIC:                             2965.
Df Model:                          40                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
symboling               109.1170    276.382      0.395      0.694    -438.499     656.733
wheelbase                10.2840    126.328      0.081      0.935    -240.019     260.587
carlength               -35.9765     59.571     -0.604      0.547    -154.008      82.055
carwidth                662.2655    295.996      2.237      0.027      75.787    1248.744
carheight                94.8147    167.784      0.565      0.573    -237.629     427.258
curbweight                2.6479      2.067      1.281      0.203      -1.447       6.743
enginesize              120.0677     30.004      4.002      0.000      60.619     179.516
boreratio              1542.5308   2066.498      0.746      0.457   -2551.970    5637.032
stroke                -5548.9926   1180.916     -4.699      0.000   -7888.826   -3209.160
compressionratio      -1041.9097    652.432     -1.597      0.113   -2334.619     250.800
horsepower               -4.4570     26.863     -0.166      0.869     -57.682      48.768
peakrpm                   2.8627      0.800      3.577      0.001       1.277       4.448
citympg                -127.0956    180.118     -0.706      0.482    -483.976     229.785
highwaympg              158.5982    162.218      0.978      0.330    -162.816     480.012
fueltype_diesel        1507.0279   3793.846      0.397      0.692   -6009.991    9024.047
fueltype_gas           -1.24e+04   3608.897     -3.436      0.001   -1.96e+04   -5248.868
aspiration_std        -6537.0947   2640.033     -2.476      0.015   -1.18e+04   -1306.209
aspiration_turbo      -4355.3134   2515.516     -1.731      0.086   -9339.487     628.860
doornumber_four       -5494.2473   2591.543     -2.120      0.036   -1.06e+04    -359.436
doornumber_two        -5398.1608   2518.441     -2.143      0.034   -1.04e+04    -408.193
carbody_convertible     -75.1688   1480.676     -0.051      0.960   -3008.939    2858.601
carbody_hardtop       -2054.9482   1520.798     -1.351      0.179   -5068.214     958.317
carbody_hatchback     -3259.6968   1187.762     -2.744      0.007   -5613.095    -906.298
carbody_sedan         -2269.9301   1264.775     -1.795      0.075   -4775.920     236.060
carbody_wagon         -3232.6641   1498.671     -2.157      0.033   -6202.088    -263.240
drivewheel_4wd        -3510.8126   1763.314     -1.991      0.049   -7004.594     -17.031
drivewheel_fwd        -3772.0201   1929.889     -1.955      0.053   -7595.849      51.809
drivewheel_rwd        -3609.5754   1849.383     -1.952      0.053   -7273.890      54.739
enginelocation_front  -1.089e+04   5053.495     -2.155      0.033   -2.09e+04    -879.555
enginetype_dohc         170.8373   1451.023      0.118      0.906   -2704.179    3045.854
enginetype_dohcv      -7710.2249   4639.931     -1.662      0.099   -1.69e+04    1483.204
enginetype_l          -

In [19]:
## To Evaluate on Test - Get Predicts
test_preds = result.predict(X_test_df)

ValueError: shapes (52,52) and (51,) not aligned: 52 (dim 1) != 51 (dim 0)

### NOTE TO TAs: there is an error here that students may run into

- `sm.add_constant` checks for ANY column that already has a single value (is constant), and if it there is one, it doesn't add anything.
- A sparse one hot encoded feature MAY wind up being a constant value in the training data. 
    - To fix this error, add `has_constant='add'` to sm.add_constant and run the model again

In [21]:
X_train_df = sm.add_constant(X_train_tf, has_constant='add', prepend=False)
X_test_df = sm.add_constant(X_test_tf, has_constant='add', prepend=False)

In [22]:
## Fit OLS Model 
model = sm.OLS(y_train,X_train_df)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.933
Model:                            OLS   Adj. R-squared:                  0.909
Method:                 Least Squares   F-statistic:                     38.91
Date:                Sat, 07 May 2022   Prob (F-statistic):           1.59e-49
Time:                        18:56:48   Log-Likelihood:                -1379.1
No. Observations:                 153   AIC:                             2840.
Df Residuals:                     112   BIC:                             2965.
Df Model:                          40                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
symboling               109.1170    276.382      0.395      0.694    -438.499     656.733
wheelbase                10.2840    126.328      0.081      0.935    -240.019     260.587
carlength               -35.9765     59.571     -0.604      0.547    -154.008      82.055
carwidth                662.2655    295.996      2.237      0.027      75.787    1248.744
carheight                94.8147    167.784      0.565      0.573    -237.629     427.258
curbweight                2.6479      2.067      1.281      0.203      -1.447       6.743
enginesize              120.0677     30.004      4.002      0.000      60.619     179.516
boreratio              1542.5308   2066.498      0.746      0.457   -2551.970    5637.032
stroke                -5548.9926   1180.916     -4.699      0.000   -7888.826   -3209.160
compressionratio      -1041.9097    652.432     -1.597      0.113   -2334.619     250.800
horsepower               -4.4570     26.863     -0.166      0.869     -57.682      48.768
peakrpm                   2.8627      0.800      3.577      0.001       1.277       4.448
citympg                -127.0956    180.118     -0.706      0.482    -483.976     229.785
highwaympg              158.5982    162.218      0.978      0.330    -162.816     480.012
fueltype_diesel        2378.2546   3522.002      0.675      0.501   -4600.141    9356.650
fueltype_gas          -1.087e+04   3184.575     -3.415      0.001   -1.72e+04   -4564.962
aspiration_std        -5339.1580   2097.483     -2.546      0.012   -9495.052   -1183.264
aspiration_turbo      -3157.3767   1974.510     -1.599      0.113   -7069.615     754.861
doornumber_four       -4296.3106   2043.350     -2.103      0.038   -8344.947    -247.674
doornumber_two        -4200.2241   1970.759     -2.131      0.035   -8105.031    -295.417
carbody_convertible     404.0059   1407.984      0.287      0.775   -2385.735    3193.746
carbody_hardtop       -1575.7735   1366.750     -1.153      0.251   -4283.813    1132.266
carbody_hatchback     -2780.5221    997.819     -2.787      0.006   -4757.572    -803.472
carbody_sedan         -1790.7554   1077.132     -1.663      0.099   -3924.955     343.444
carbody_wagon         -2753.4895   1324.994     -2.078      0.040   -5378.795    -128.184
drivewheel_4wd        -2712.1881   1459.567     -1.858      0.066   -5604.133     179.756
drivewheel_fwd        -2973.3956   1577.656     -1.885      0.062   -6099.318     152.527
drivewheel_rwd        -2810.9509   1517.121     -1.853      0.067   -5816.931     195.029
enginelocation_front  -8496.5346   3941.938     -2.155      0.033   -1.63e+04    -686.090
enginetype_dohc         513.1049   1380.817      0.372      0.711   -2222.807    3249.017
enginetype_dohcv      -7367.9573   4600.079     -1.602      0.112   -1.65e+04    1746.510
enginetype_l           

In [31]:
## To Evaluate on Test - Get Predicts
test_preds = result.predict(X_test_df)
test_r2 = r2_score(y_test, test_preds)
test_mse = mean_squared_error(y_test, test_preds)

print(f'The testing r-square value is {test_r2:.2f} and the testing mean squared error is {test_mse:,.2f}.')


## To Evaluate on Test - Get Predicts
train_preds = result.predict(X_train_df)
train_r2 = r2_score(y_train, train_preds)
train_mse = mean_squared_error(y_train, train_preds)

print(f'The training r-square value is {train_r2:.2f} and the testing mean squared error is {train_mse:,.2f}.')


The testing r-square value is 0.94 and the testing mean squared error is 4,932,689.37.
The training r-square value is 0.93 and the testing mean squared error is 3,952,848.72.
